In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import seaborn as sns
import math
import gc

import sys
from pathlib import Path
d = Path().resolve().parent.parent
sys.path.insert(0, str(d))
import util.utils as utils
import util.plot_utils as plot_utils



plt.rcParams["figure.figsize"] = (12,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

from catboost import Pool, CatBoostRegressor

InteractiveShell.ast_node_interactivity = "all"
np.set_printoptions(precision=2)

ModuleNotFoundError: No module named 'catboost'

In [5]:
project_name="click-prediction"
def get_file_location(filename):
    return "../data/%s/%s" % (project_name,filename)

In [6]:
def fast_read_and_append(file_path,chunksize,fullsize):
    # in chunk reading be careful as pandas might infer a columns dtype as different for diff chunk
    # As such specifying a dtype while reading by giving params to read_csv maybe better
    # Label encoding will fail if half the rows for same column is int and rest are str
    # In case of that already happened then df_test["publisherId"] = df_test["publisherId"].apply(str)
    df = pd.DataFrame()
    iterations = 0
    total_needed_iters = math.ceil(fullsize/chunksize)
    for x in pd.read_csv(file_path, chunksize=chunksize,low_memory=False):
        print("iterations= %s out of %s" %  (iterations,total_needed_iters))
        df = pd.concat([df, x], ignore_index=True)
        iterations += 1
        gc.collect()
    return df

In [7]:
df=fast_read_and_append("../data/%s/train-step-1.csv" % project_name,1000000,63367217)

iterations= 0 out of 64
iterations= 1 out of 64
iterations= 2 out of 64
iterations= 3 out of 64
iterations= 4 out of 64
iterations= 5 out of 64
iterations= 6 out of 64
iterations= 7 out of 64
iterations= 8 out of 64
iterations= 9 out of 64
iterations= 10 out of 64
iterations= 11 out of 64
iterations= 12 out of 64
iterations= 13 out of 64
iterations= 14 out of 64
iterations= 15 out of 64
iterations= 16 out of 64
iterations= 17 out of 64
iterations= 18 out of 64
iterations= 19 out of 64
iterations= 20 out of 64
iterations= 21 out of 64
iterations= 22 out of 64
iterations= 23 out of 64
iterations= 24 out of 64
iterations= 25 out of 64
iterations= 26 out of 64
iterations= 27 out of 64
iterations= 28 out of 64
iterations= 29 out of 64
iterations= 30 out of 64
iterations= 31 out of 64
iterations= 32 out of 64
iterations= 33 out of 64
iterations= 34 out of 64
iterations= 35 out of 64
iterations= 36 out of 64
iterations= 37 out of 64
iterations= 38 out of 64
iterations= 39 out of 64
iterations

In [8]:
df_test=fast_read_and_append("../data/%s/test-step-1.csv" % project_name,300000,24510750)

iterations= 0 out of 82
iterations= 1 out of 82
iterations= 2 out of 82
iterations= 3 out of 82
iterations= 4 out of 82
iterations= 5 out of 82
iterations= 6 out of 82
iterations= 7 out of 82
iterations= 8 out of 82
iterations= 9 out of 82
iterations= 10 out of 82
iterations= 11 out of 82
iterations= 12 out of 82
iterations= 13 out of 82
iterations= 14 out of 82
iterations= 15 out of 82
iterations= 16 out of 82
iterations= 17 out of 82
iterations= 18 out of 82
iterations= 19 out of 82
iterations= 20 out of 82
iterations= 21 out of 82
iterations= 22 out of 82
iterations= 23 out of 82
iterations= 24 out of 82
iterations= 25 out of 82
iterations= 26 out of 82
iterations= 27 out of 82
iterations= 28 out of 82
iterations= 29 out of 82
iterations= 30 out of 82
iterations= 31 out of 82
iterations= 32 out of 82
iterations= 33 out of 82
iterations= 34 out of 82
iterations= 35 out of 82
iterations= 36 out of 82
iterations= 37 out of 82
iterations= 38 out of 82
iterations= 39 out of 82
iterations

In [9]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [10]:
df_test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [11]:
df["conv_status"] = 0
df.loc[df["ConversionStatus"],"conv_status"] = 1
df.drop(['ConversionStatus'], axis = 1, inplace = True)

In [12]:
df_test["ClickDate"] = pd.to_datetime(df_test["ClickDate"])


In [13]:
df["ClickDate"] = pd.to_datetime(df["ClickDate"])

In [14]:
df_test["dayofweek"] = df_test["ClickDate"].map(lambda x: x.dayofweek)
df_test["hour"] = df_test["ClickDate"].map(lambda x: x.hour)
df_test["dayofyear"] = df_test["ClickDate"].map(lambda x: x.dayofyear)

In [15]:
df_test.drop(['ClickDate'], axis = 1, inplace = True)

In [16]:
df["dayofweek"] = df["ClickDate"].map(lambda x: x.dayofweek)
df["hour"] = df["ClickDate"].map(lambda x: x.hour)
df["dayofyear"] = df["ClickDate"].map(lambda x: x.dayofyear)

In [17]:

df.drop(['ClickDate'], axis = 1, inplace = True)

In [18]:

df.drop(['ConversionDate'], axis = 1, inplace = True)

In [20]:
df_test.to_csv(get_file_location("test-step-2.csv"),index=False)

In [21]:
df.to_csv(get_file_location("train-step-2.csv") ,index=False)

In [19]:
df.dtypes

ID                        int64
Country                   int64
Carrier                   int64
TrafficType               int64
Device                    int64
Browser                   int64
OS                        int64
RefererUrl                int64
UserIp                    int64
ConversionPayOut        float64
publisherId               int64
subPublisherId            int64
advertiserCampaignId      int64
Fraud                     int64
conv_status               int64
dayofweek                 int64
hour                      int64
dayofyear                 int64
dtype: object